In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

import tensorflow as tf

from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Dense, Flatten
from keras.optimizers import Adam
from keras.regularizers import L2
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler

from scipy.stats import pointbiserialr, permutation_test

In [2]:
stresses = ['Gm', 'Drought', 'Nutrient_Deficiency', 'Fs', 'Salinity']

In [3]:
csv_path = r'..\combined.csv'
df = pd.read_csv(csv_path)
df.drop(columns=['Unnamed: 0'], inplace=True, errors='ignore')
df.drop(columns=['Fungal_infection'], inplace=True, errors='ignore')
df[stresses] = df[stresses].astype(bool)

C:\Users\amitt\AppData\Local\Temp\ipykernel_14984\1297277926.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


In [4]:
spec_cols = [col for col in df.columns if col[0] == 'X']

In [5]:
df

,Experiment,Time,Species,Soil,Gm,Drought,Nutrient_Deficiency,Fs,Salinity,X350,...,B_carotene,Glucose,Fructose,Sucrose,Sugars,Starch,Ellagic,Gal,Rut,CTs
0,2018_EEL,0,Black Walnut,F,False,False,False,False,False,0.0995,...,180.672913,4.114120,4.321044,2.106877,10.377295,2.430284,215.327766,123.012942,37.911137,16.153354
1,2018_EEL,0,Black Walnut,F,False,False,False,False,False,0.0995,...,185.691351,5.253359,4.398900,2.090093,10.430779,1.620732,159.648059,140.224392,34.913800,12.904982
2,2018_EEL,0,Black Walnut,F,False,False,False,False,False,0.0995,...,223.876583,5.130809,4.122389,2.148208,9.001072,1.204313,169.093819,137.268720,26.315744,13.429072
3,2018_EEL,0,Black Walnut,F,False,False,False,False,False,0.0995,...,134.435161,6.996149,5.455238,1.557863,14.307212,5.177277,98.561955,70.040873,15.097144,13.590456
4,2018_EEL,0,Black Walnut,F,False,False,False,False,False,0.0995,...,160.339162,6.490596,3.602786,2.163037,10.592087,1.899691,244.166164,169.102470,40.167820,18.469889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23651,2019_Martell,2,Red Oak,NaN,False,False,False,False,True,0.1006,...,50.680347,0.491981,2.142298,2.022112,6.362280,0.064974,229.208398,46.702728,30.452206,2.524192
23652,2019_Martell,2,Red Oak,NaN,False,False,False,False,True,0.1006,...,2.102093,3.317779,2.946461,4.029103,8.485325,0.677890,228.950841,102.741536,46.396327,6.360440
23653,2019_Martell,2,Red Oak,NaN,False,False,False,False,True,0.1006,...,7.064565,1.433723,1.722290,1.883972,5.001576,-2.390555,439.018157,102.577759,34.947248,9.521822
23654,2019_Martell,2,Red Oak,NaN,False,False,False,False,True,0.1006,...,11.266166,0.839294,2.183542,2.365718,5.722241,0.596050,311.728885,39.374667,34.287740,1.880939
